In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
df.isnull().values.any()

np.False_

In [4]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)

LogisticRegression


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 1, 10, 100],
    'solver': ['saga'],
    'max_iter': [50,100],
    'l1_ratio': [0.1, 0.5, 0.7, 1]
}
grid=GridSearchCV(LogisticRegression(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'l1_ratio': [0.1, 0.5, 0.7, 1],
                         'max_iter': [50, 100],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['saga']},
             scoring='f1_weighted', verbose=3)

In [7]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[  0 150]
 [  0 249]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       150
           1       0.62      1.00      0.77       249

    accuracy                           0.62       399
   macro avg       0.31      0.50      0.38       399
weighted avg       0.39      0.62      0.48       399

The f1_macro value for the best parameters {'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 50, 'penalty': 'l1', 'solver': 'saga'}: 0.4796017822333612
The roc_score: 0.7295046854082999


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:153

In [8]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_l1_ratio,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.041829,0.015710,0.011899,0.003220,0.1,0.1,50,l1,saga,"{'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 50, 'p...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
1,0.038714,0.012093,0.009392,0.002287,0.1,0.1,50,l2,saga,"{'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 50, 'p...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
2,0.031914,0.003722,0.021430,0.018198,0.1,0.1,50,elasticnet,saga,"{'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 50, 'p...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
3,0.060506,0.015927,0.013903,0.014389,0.1,0.1,100,l1,saga,"{'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 100, '...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
4,0.040896,0.016243,0.006379,0.001753,0.1,0.1,100,l2,saga,"{'C': 0.1, 'l1_ratio': 0.1, 'max_iter': 100, '...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.018158,0.005261,0.006224,0.003037,100.0,1.0,50,l2,saga,"{'C': 100, 'l1_ratio': 1, 'max_iter': 50, 'pen...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
92,0.021372,0.004170,0.004311,0.000400,100.0,1.0,50,elasticnet,saga,"{'C': 100, 'l1_ratio': 1, 'max_iter': 50, 'pen...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
93,0.031597,0.005590,0.004704,0.001039,100.0,1.0,100,l1,saga,"{'C': 100, 'l1_ratio': 1, 'max_iter': 100, 'pe...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
94,0.021242,0.008844,0.004010,0.000837,100.0,1.0,100,l2,saga,"{'C': 100, 'l1_ratio': 1, 'max_iter': 100, 'pe...",0.480769,0.480769,0.480769,0.480769,0.474881,0.479592,0.002355,1
